In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# **Additional Data Cleaning**

In [ ]:
# Mount drive
# drive.mount('/content/drive/MyDrive/Computer Vision Project')

# Copy the zip file to Colab's local storage
!cp "/content/drive/MyDrive/LocalCV/Copy of Final Data.zip" /content/

# Create a destination directory and extract the dataset
!mkdir -p /content/dataset
!unzip "/content/Copy of Final Data.zip" -d /content/dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/Final Data/train/Bentley_Mulsanne_Sedan_2011/02386.jpg  
  inflating: /content/dataset/Final Data/train/Bentley_Mulsanne_Sedan_2011/02587.jpg  
  inflating: /content/dataset/Final Data/train/Bentley_Mulsanne_Sedan_2011/04690.jpg  
  inflating: /content/dataset/Final Data/train/Bentley_Mulsanne_Sedan_2011/03285.jpg  
  inflating: /content/dataset/Final Data/train/2013-2016_chevrolet_silverado_1500_ld/5834.jpg  
  inflating: /content/dataset/Final Data/train/Bentley_Mulsanne_Sedan_2011/04783.jpg  
  inflating: /content/dataset/Final Data/train/2013-2016_chevrolet_silverado_1500_ld/4644.jpg  
  inflating: /content/dataset/Final Data/train/2013-2016_chevrolet_silverado_1500_ld/5830.jpg  
  inflating: /content/dataset/Final Data/train/Bentley_Mulsanne_Sedan_2011/06561.jpg  
  inflating: /content/dataset/Final Data/train/2013-2016_chevrolet_silverado_1500_ld/11884.jpg  
  inflating: /content/dataset/Final Data/t

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from google.colab import drive
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import shutil
from tqdm import tqdm

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 14.9 MB/s eta 0:00:00


In [ ]:
def filter_car_images(source_dir, target_dir, model_path, confidence_threshold=0.9):
    """
    Filters images in a nested directory, keeping only those with detected cars.
    """
    # Load YOLO model
    model = YOLO(model_path)

    # Ensure target directory exists
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Walk through all files in the nested directory
    for root, _, files in os.walk(source_dir):
        for file in tqdm(files):
            image_path = os.path.join(root, file)
            try:
                # Check if the file is an image
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    # Run YOLO model on the image
                    results = model.predict(source=image_path, verbose=False)

                    # Extract detection confidences
                    confidences = [box.conf.cpu().item() for box in results[0].boxes]

                    # Check if any detection is above the confidence threshold
                    if any(conf >= confidence_threshold for conf in confidences):
                        # Copy image to target directory if a car is detected
                        class_dir = os.path.basename(root)  # Use subfolder name as class
                        target_class_dir = os.path.join(target_dir, class_dir)

                        # Ensure class subdirectory exists in the target directory
                        if not os.path.exists(target_class_dir):
                            os.makedirs(target_class_dir)

                        shutil.copy(image_path, os.path.join(target_class_dir, file))
            except Exception as e:
                print(f"Error processing {image_path}: {e}")


In [ ]:
train_dir = '/content/dataset/Final Data/train'
val_dir = '/content/dataset/Final Data/val'
test_dir = '/content/dataset/Final Data/test'
# Paths to directories
model_path = "/content/drive/MyDrive/LocalCV/yolov8_car_detection_ks.pt"

# Apply filtering for train, val, and test directories
filter_car_images(train_dir, "/content/dataset/Filtered/train", model_path)
filter_car_images(val_dir, "/content/dataset/Filtered/val", model_path)
filter_car_images(test_dir, "/content/dataset/Filtered/test", model_path)

0it [00:00, ?it/s]
100%|██████████| 49/49 [00:02<00:00, 16.94it/s]
0it [00:00, ?it/s]
100%|██████████| 26/26 [00:01<00:00, 19.47it/s]
0it [00:00, ?it/s]
100%|██████████| 13/13 [00:00<00:00, 23.75it/s]
0it [00:00, ?it/s]
100%|██████████| 7/7 [00:00<00:00, 20.01it/s]
0it [00:00, ?it/s]
100%|██████████| 16/16 [00:00<00:00, 20.33it/s]
0it [00:00, ?it/s]
100%|██████████| 9/9 [00:00<00:00, 18.05it/s]


In [ ]:
# Updated filtered paths
train_dir = '/content/dataset/Filtered/train'
val_dir = '/content/dataset/Filtered/val'
test_dir = '/content/dataset/Filtered/test'

# Check training classes
train_classes = os.listdir(train_dir)
print(f"Number of training classes: {len(train_classes)}")
print(f"Training Classes: {train_classes[:10]}")

# Check validation classes
val_classes = os.listdir(val_dir)
print(f"Number of validation classes: {len(val_classes)}")
print(f"Validation Classes: {val_classes[:10]}")

# Check test classes
test_classes = os.listdir(test_dir)
print(f"Number of test classes: {len(test_classes)}")
print(f"Test Classes: {test_classes[:10]}")


Number of training classes: 391
Training Classes: ['Aston_Martin_V8_Vantage_Coupe_2012', 'Audi_V8_Sedan_1994', 'Chevrolet_Corvette_Ron_Fellows_Edition_Z06_2007', '2017-2020_nissan_murano', '2013-2016_ford_mustang', 'Jeep_Liberty_SUV_2012', 'Suzuki_Aerio_Sedan_2007', '2013-2016_gmc_acadia', 'Land_Rover_LR2_SUV_2012', '2013-2016_nissan_nv200']
Number of validation classes: 391
Validation Classes: ['Aston_Martin_V8_Vantage_Coupe_2012', 'Audi_V8_Sedan_1994', 'Chevrolet_Corvette_Ron_Fellows_Edition_Z06_2007', '2017-2020_nissan_murano', '2013-2016_ford_mustang', 'Jeep_Liberty_SUV_2012', 'Suzuki_Aerio_Sedan_2007', '2013-2016_gmc_acadia', 'Land_Rover_LR2_SUV_2012', '2013-2016_nissan_nv200']
Number of test classes: 391
Test Classes: ['Aston_Martin_V8_Vantage_Coupe_2012', 'Audi_V8_Sedan_1994', 'Chevrolet_Corvette_Ron_Fellows_Edition_Z06_2007', '2017-2020_nissan_murano', '2013-2016_ford_mustang', 'Jeep_Liberty_SUV_2012', 'Suzuki_Aerio_Sedan_2007', '2013-2016_gmc_acadia', 'Land_Rover_LR2_SUV_2012'

In [ ]:
import os
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Get class names from training directory (all directories should have same classes)
train_classes = sorted(os.listdir(train_dir))

# Convert to DataFrame
df = pd.DataFrame(train_classes, columns=['class_name'])

# Save to CSV in Google Drive
csv_path = '/content/drive/My Drive/class_names.csv'
df.to_csv(csv_path, index=False)

print(f"Saved {len(train_classes)} class names to {csv_path}")
print("First 10 classes:", train_classes[:10])

# **RESNET50 BEST MODEL**

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Base directory for saving models
BASE_SAVE_DIR = "/content/drive/My Drive/LocalCV"
os.makedirs(BASE_SAVE_DIR, exist_ok=True)

# Custom callback for monitoring
class DataMonitorCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\nStarting epoch {epoch+1}")

    def on_epoch_end(self, epoch, logs=None):
        print(f"\nEpoch {epoch+1} complete")
        print(f"Training accuracy: {logs.get('accuracy', 'N/A'):.4f}")
        print(f"Validation accuracy: {logs.get('val_accuracy', 'N/A'):.4f}")

# Data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input_resnet50,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input_resnet50
)

# Create generators with repeatable data
def make_generator_repeatable(generator):
    while True:
        for x, y in generator:
            yield x, y

print("Loading data generators...")
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

valid_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Calculate steps properly with integer conversion
STEPS_PER_EPOCH = train_generator.n // BATCH_SIZE
VALIDATION_STEPS = int(np.ceil(valid_generator.n / BATCH_SIZE))

# Get number of classes
num_classes = len(train_generator.class_indices)
print(f"Number of classes detected: {num_classes}")

# Create wrapped generators
train_gen = make_generator_repeatable(train_generator)
valid_gen = make_generator_repeatable(valid_generator)

def create_resnet50_model():
    # Create base model
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )

    # Freeze all layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)

    return model

def train_resnet50_model(model, save_dir="ResNet50_v2"):
    save_dir = os.path.join(BASE_SAVE_DIR, save_dir)
    os.makedirs(save_dir, exist_ok=True)

    # Print training configuration
    print("\nTraining Configuration:")
    print(f"Total training samples: {train_generator.n}")
    print(f"Total validation samples: {valid_generator.n}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Steps per epoch: {STEPS_PER_EPOCH}")
    print(f"Validation steps: {VALIDATION_STEPS}")

    # Callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    )

    lr_scheduler = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-8,
        verbose=1
    )

    log_dir = os.path.join(save_dir, "logs/fit/" + pd.Timestamp.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    monitor_callback = DataMonitorCallback()

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train model
    history = model.fit(
        train_gen,
        steps_per_epoch=STEPS_PER_EPOCH,
        validation_data=valid_gen,
        validation_steps=VALIDATION_STEPS,
        epochs=50,
        callbacks=[early_stopping, lr_scheduler, tensorboard_callback, monitor_callback]
    )

    # Save models
    model.save(os.path.join(save_dir, 'car_classifier_resnet50_v2_final.h5'), save_format='h5')
    print(f"Model saved to {save_dir}")

    return history, model

# Train model with new names
print("Creating and training ResNet50 model...")
resnet50_model = create_resnet50_model()
resnet50_history, trained_resnet50 = train_resnet50_model(resnet50_model)

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(resnet50_history.history['accuracy'], label='Training Accuracy')
plt.plot(resnet50_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('ResNet50 Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(resnet50_history.history['loss'], label='Training Loss')
plt.plot(resnet50_history.history['val_loss'], label='Validation Loss')
plt.title('ResNet50 Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(BASE_SAVE_DIR, 'ResNet50_v2', 'resnet50_v2_training_history.png'))
plt.close()


Loading data generators...
Found 18396 images belonging to 391 classes.
Found 4798 images belonging to 391 classes.
Number of classes detected: 391
Creating and training ResNet50 model...

Training Configuration:
Total training samples: 18396
Total validation samples: 4798
Batch size: 32
Steps per epoch: 574
Validation steps: 150

Starting epoch 1
Epoch 1/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 0.0149 - loss: 5.9443
Epoch 1 complete
Training accuracy: 0.0260
Validation accuracy: 0.0982
574/574 ━━━━━━━━━━━━━━━━━━━━ 249s 413ms/step - accuracy: 0.0149 - loss: 5.9435 - val_accuracy: 0.0982 - val_loss: 4.1814 - learning_rate: 0.0010

Starting epoch 2
Epoch 2/50
574/574 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 0.0615 - loss: 4.6205
Epoch 2 complete
Training accuracy: 0.0695
Validation accuracy: 0.1511
574/574 ━━━━━━━━━━━━━━━━━━━━ 235s 402ms/step - accuracy: 0.0615 - loss: 4.6203 - val_accuracy: 0.1511 - val_loss: 3.6458 - learning_rate: 0.0010

Starting epoch 3
Epoch 

Model saved to /content/drive/My Drive/LocalCV/ResNet50_v2


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Base directory for saving models
BASE_SAVE_DIR = "/content/drive/My Drive/LocalCV"
os.makedirs(BASE_SAVE_DIR, exist_ok=True)

# Custom callback for monitoring
class DataMonitorCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\nStarting epoch {epoch+1}")

    def on_epoch_end(self, epoch, logs=None):
        print(f"\nEpoch {epoch+1} complete")
        print(f"Training accuracy: {logs.get('accuracy', 'N/A'):.4f}")
        print(f"Validation accuracy: {logs.get('val_accuracy', 'N/A'):.4f}")

# Data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input_resnet50,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input_resnet50
)

# Create generators with repeatable data
def make_generator_repeatable(generator):
    while True:
        for x, y in generator:
            yield x, y

print("Loading data generators...")
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

valid_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Calculate steps properly with integer conversion
STEPS_PER_EPOCH = train_generator.n // BATCH_SIZE
VALIDATION_STEPS = int(np.ceil(valid_generator.n / BATCH_SIZE))

# Get number of classes
num_classes = len(train_generator.class_indices)
print(f"Number of classes detected: {num_classes}")

# Create wrapped generators
train_gen = make_generator_repeatable(train_generator)
valid_gen = make_generator_repeatable(valid_generator)

def create_resnet50_model_enhanced():
    # Create base model
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )

    # Freeze all layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)

    return model

def create_resnet50_model_optimized():
    # Create base model
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )

    # Freeze all layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers with modified architecture
    x = GlobalAveragePooling2D()(base_model.output)

    # First dense block - reduced dropout and slightly smaller layer
    x = Dense(1536, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)  # Start with lower dropout

    # Second dense block
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    # Third dense block
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    # Fourth dense block
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    # Final dense block - increased from 128 to 192
    x = Dense(192, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)

    return model

def train_resnet50_model_optimized(model, save_dir="ResNet50_optimized"):
    save_dir = os.path.join(BASE_SAVE_DIR, save_dir)
    os.makedirs(save_dir, exist_ok=True)

    # Print training configuration
    print("\nTraining Configuration:")
    print(f"Total training samples: {train_generator.n}")
    print(f"Total validation samples: {valid_generator.n}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Steps per epoch: {STEPS_PER_EPOCH}")
    print(f"Validation steps: {VALIDATION_STEPS}")

    # Modified callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,  # Keep patience higher for early stopping
        restore_best_weights=True,
        verbose=1
    )

    # More aggressive learning rate reduction
    lr_scheduler = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,  # More aggressive reduction (0.3 instead of 0.5)
        patience=2,  # Reduced patience as requested
        min_lr=1e-8,
        verbose=1
    )

    log_dir = os.path.join(save_dir, "logs/fit/" + pd.Timestamp.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    monitor_callback = DataMonitorCallback()

    # Custom learning rate schedule with warmup
    initial_learning_rate = 0.001  # Back to original learning rate

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=initial_learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train model
    history = model.fit(
        train_gen,
        steps_per_epoch=STEPS_PER_EPOCH,
        validation_data=valid_gen,
        validation_steps=VALIDATION_STEPS,
        epochs=50,
        callbacks=[early_stopping, lr_scheduler, tensorboard_callback, monitor_callback]
    )

    # Save models
    model.save(os.path.join(save_dir, 'car_classifier_resnet50_optimized_final.h5'), save_format='h5')
    print(f"Model saved to {save_dir}")

    return history, model

# Train model with optimized settings
print("Creating and training optimized ResNet50 model...")
resnet50_model = create_resnet50_model_optimized()
resnet50_history, trained_resnet50 = train_resnet50_model_optimized(resnet50_model)

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(resnet50_history.history['accuracy'], label='Training Accuracy')
plt.plot(resnet50_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Optimized ResNet50 Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(resnet50_history.history['loss'], label='Training Loss')
plt.plot(resnet50_history.history['val_loss'], label='Validation Loss')
plt.title('Optimized ResNet50 Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(BASE_SAVE_DIR, 'ResNet50_optimized', 'resnet50_optimized_training_history.png'))
plt.close()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 26,351,754 (100.52 MB)

 Trainable params: 2,760,458 (10.53 MB)

 Non-trainable params: 23,591,296 (89.99 MB)

In [ ]:
# Create test data generator
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input_resnet50
)

# Create test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Calculate test steps
TEST_STEPS = int(np.ceil(test_generator.n / 32))


Found 5932 images belonging to 391 classes.


# **Evaluation on test Data**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from collections import Counter

# Load the saved model - updated path to match your save location
model_path = "/content/drive/My Drive/LocalCV/ResNet50_v2/car_classifier_resnet50_v2_final.h5"
loaded_model = load_model(model_path)

# Get predictions using the test generator we created earlier
print("Generating predictions...")
predictions = loaded_model.predict(
    test_generator,
    steps=TEST_STEPS,
    verbose=1
)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Get class names
class_names = list(test_generator.class_indices.keys())
class_indices = {v: k for k, v in test_generator.class_indices.items()}

# Create confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Calculate per-class accuracy
per_class_accuracy = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
class_performance = pd.DataFrame({
    'Class': class_names,
    'Accuracy': per_class_accuracy,
    'Total Samples': conf_matrix.sum(axis=1),
    'Correct Predictions': conf_matrix.diagonal(),
    'Incorrect Predictions': conf_matrix.sum(axis=1) - conf_matrix.diagonal()
})

# Sort by accuracy to get top and bottom performing classes
top_25_classes = class_performance.nlargest(25, 'Accuracy')
bottom_25_classes = class_performance.nsmallest(25, 'Accuracy')

# Save detailed results to a file
results_path = os.path.join(BASE_SAVE_DIR, 'ResNet50_enhanced', 'detailed_evaluation_results.txt')
with open(results_path, 'w') as f:
    # Overall model performance
    f.write("="*50 + "\n")
    f.write("MODEL PERFORMANCE SUMMARY\n")
    f.write("="*50 + "\n")
    f.write(f"Overall Model Accuracy: {np.mean(per_class_accuracy):.4f}\n")
    f.write(f"Total Test Samples: {len(true_classes)}\n")
    f.write(f"Total Correct Predictions: {sum(predicted_classes == true_classes)}\n")
    f.write(f"Total Incorrect Predictions: {sum(predicted_classes != true_classes)}\n\n")

    # Top 25 best performing classes
    f.write("="*50 + "\n")
    f.write("TOP 25 BEST CLASSIFIED CARS\n")
    f.write("="*50 + "\n")
    f.write("\nFormat: Class Name (Accuracy) [Correct/Total Samples]\n")
    for _, row in top_25_classes.iterrows():
        f.write(f"{row['Class']:<40} ({row['Accuracy']:.4f}) [{int(row['Correct Predictions'])}/{int(row['Total Samples'])}]\n")

    # Top 25 worst performing classes
    f.write("\n" + "="*50 + "\n")
    f.write("TOP 25 MOST MISCLASSIFIED CARS\n")
    f.write("="*50 + "\n")
    f.write("\nFormat: Class Name (Accuracy) [Correct/Total Samples]\n")
    for _, row in bottom_25_classes.iterrows():
        f.write(f"{row['Class']:<40} ({row['Accuracy']:.4f}) [{int(row['Correct Predictions'])}/{int(row['Total Samples'])}]\n")

    # Calculate and save most common misclassifications
    confusion_pairs = []
    for i in range(len(conf_matrix)):
        for j in range(len(conf_matrix)):
            if i != j and conf_matrix[i, j] > 0:
                confusion_pairs.append({
                    'True Class': class_indices[i],
                    'Predicted Class': class_indices[j],
                    'Count': conf_matrix[i, j]
                })

    confusion_df = pd.DataFrame(confusion_pairs)
    top_confusions = confusion_df.nlargest(25, 'Count')

    f.write("\n" + "="*50 + "\n")
    f.write("TOP 25 MOST COMMON MISCLASSIFICATION PAIRS\n")
    f.write("="*50 + "\n")
    f.write("\nFormat: True Class -> Predicted Class [Number of Occurrences]\n")
    for _, row in top_confusions.iterrows():
        f.write(f"{row['True Class']:<35} -> {row['Predicted Class']:<35} [{int(row['Count'])}]\n")

# Create and save confusion matrix visualization
plt.figure(figsize=(20, 20))
sns.heatmap(conf_matrix, xticklabels=class_names, yticklabels=class_names,
            annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(os.path.join(BASE_SAVE_DIR, 'ResNet50_enhanced', 'detailed_confusion_matrix.png'))
plt.close()



Generating predictions...


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


186/186 ━━━━━━━━━━━━━━━━━━━━ 23s 108ms/step

Evaluation results have been saved to: /content/drive/My Drive/LocalCV/ResNet50_enhanced/detailed_evaluation_results.txt


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/LocalCV/ResNet50_enhanced/detailed_evaluation_results.txt'

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import os
import shutil

# Constants from your original setup
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

def split_test_directory(test_dir):
    """Split test directory into Stanford and scraped subdirectories"""
    # Create stanford and scraped subdirectories
    stanford_dir = os.path.join(os.path.dirname(test_dir), 'test_stanford')
    scraped_dir = os.path.join(os.path.dirname(test_dir), 'test_scraped')

    # Remove directories if they exist
    if os.path.exists(stanford_dir):
        shutil.rmtree(stanford_dir)
    if os.path.exists(scraped_dir):
        shutil.rmtree(scraped_dir)

    # Create new directories
    os.makedirs(stanford_dir)
    os.makedirs(scraped_dir)

    # Iterate through class folders in test directory
    for class_name in os.listdir(test_dir):
        source_dir = os.path.join(test_dir, class_name)
        if not os.path.isdir(source_dir):
            continue

        # Determine if class is Stanford or scraped
        if any(class_name.startswith(str(year)) for year in range(1900, 2024)):
            dest_dir = os.path.join(scraped_dir, class_name)
        else:
            dest_dir = os.path.join(stanford_dir, class_name)

        # Copy the class folder to appropriate destination
        shutil.copytree(source_dir, dest_dir)

    return stanford_dir, scraped_dir

def evaluate_dataset(model, generator, dataset_name, save_path):
    """Evaluate model on a specific dataset and save results"""
    # Get predictions
    steps = int(np.ceil(generator.samples / generator.batch_size))
    predictions = model.predict(generator, steps=steps, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = generator.classes[:len(predicted_classes)]

    # Calculate per-class accuracy
    class_indices = generator.class_indices
    idx_to_class = {v: k for k, v in class_indices.items()}

    per_class_accuracy = {}
    for class_idx in np.unique(true_classes):
        mask = true_classes == class_idx
        if np.sum(mask) > 0:
            class_name = idx_to_class[class_idx]
            per_class_accuracy[class_name] = np.mean(predicted_classes[mask] == true_classes[mask])

    # Convert to DataFrame for sorting
    performance_df = pd.DataFrame({
        'Class': list(per_class_accuracy.keys()),
        'Accuracy': list(per_class_accuracy.values())
    })

    # Get top and bottom 5
    top_5 = performance_df.nlargest(5, 'Accuracy')
    bottom_5 = performance_df.nsmallest(5, 'Accuracy')

    # Save results
    with open(os.path.join(save_path, f'{dataset_name}_evaluation.txt'), 'w') as f:
        f.write(f"===== {dataset_name.upper()} DATASET EVALUATION =====\n\n")
        f.write(f"Total Classes: {len(class_indices)}\n")
        f.write(f"Total Samples: {len(true_classes)}\n")
        f.write(f"Overall Accuracy: {np.mean(predicted_classes == true_classes):.4f}\n\n")

        f.write("TOP 5 BEST CLASSIFIED CARS:\n")
        f.write("===========================\n")
        for _, row in top_5.iterrows():
            f.write(f"{row['Class']:<40} ({row['Accuracy']:.4f})\n")

        f.write("\nBOTTOM 5 WORST CLASSIFIED CARS:\n")
        f.write("==============================\n")
        for _, row in bottom_5.iterrows():
            f.write(f"{row['Class']:<40} ({row['Accuracy']:.4f})\n")

# Main execution
print("Splitting test directory...")
test_dir = '/content/dataset/Filtered/test'
stanford_dir, scraped_dir = split_test_directory(test_dir)

# Create data generator with same preprocessing as original
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input_resnet50
)

# Create Stanford generator
print("Creating Stanford generator...")
stanford_generator = test_datagen.flow_from_directory(
    stanford_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Create scraped generator
print("Creating scraped generator...")
scraped_generator = test_datagen.flow_from_directory(
    scraped_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Load model
print("Loading model...")
model_path = "/content/drive/My Drive/LocalCV/ResNet50_v2/car_classifier_resnet50_v2_final.h5"
loaded_model = load_model(model_path)

# Create save directory if it doesn't exist
save_path = "/content/drive/My Drive/LocalCV/ResNet50_v2"
os.makedirs(save_path, exist_ok=True)

# Evaluate both datasets
print("Evaluating Stanford dataset...")
evaluate_dataset(loaded_model, stanford_generator, "stanford", save_path)

print("Evaluating scraped dataset...")
evaluate_dataset(loaded_model, scraped_generator, "scraped", save_path)

print("\nEvaluation completed! Results saved to:")
print(f"{save_path}/stanford_evaluation.txt")
print(f"{save_path}/scraped_evaluation.txt")

# Clean up temporary directories
print("Cleaning up...")
shutil.rmtree(stanford_dir)
shutil.rmtree(scraped_dir)

Splitting test directory...
Creating Stanford generator...
Found 3264 images belonging to 195 classes.
Creating scraped generator...
Found 2668 images belonging to 196 classes.
Loading model...


Evaluating Stanford dataset...


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


102/102 ━━━━━━━━━━━━━━━━━━━━ 17s 102ms/step
Evaluating scraped dataset...


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


84/84 ━━━━━━━━━━━━━━━━━━━━ 15s 142ms/step

Evaluation completed! Results saved to:
/content/drive/My Drive/LocalCV/ResNet50_v2/stanford_evaluation.txt
/content/drive/My Drive/LocalCV/ResNet50_v2/scraped_evaluation.txt
Cleaning up...


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import os

def evaluate_with_class_types(model, generator, save_path):
    """
    Evaluate model on the full test generator but analyze Stanford and scraped classes separately
    """
    # Get predictions for full dataset
    steps = int(np.ceil(generator.samples / generator.batch_size))
    predictions = model.predict(generator, steps=steps, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = generator.classes[:len(predicted_classes)]

    # Get class mappings
    class_indices = generator.class_indices
    idx_to_class = {v: k for k, v in class_indices.items()}

    # Separate classes into Stanford and scraped
    stanford_class_indices = []
    scraped_class_indices = []

    for idx, class_name in idx_to_class.items():
        if any(class_name.startswith(str(year)) for year in range(1900, 2024)):
            scraped_class_indices.append(idx)
        else:
            stanford_class_indices.append(idx)

    def analyze_class_set(class_indices, name):
        # Get mask for samples belonging to this set
        mask = np.isin(true_classes, class_indices)
        set_true = true_classes[mask]
        set_pred = predicted_classes[mask]

        # Calculate per-class accuracy
        per_class_accuracy = {}
        for class_idx in class_indices:
            class_mask = set_true == class_idx
            if np.sum(class_mask) > 0:
                class_name = idx_to_class[class_idx]
                accuracy = np.mean(set_pred[class_mask] == set_true[class_mask])
                per_class_accuracy[class_name] = accuracy

        # Convert to DataFrame
        performance_df = pd.DataFrame({
            'Class': list(per_class_accuracy.keys()),
            'Accuracy': list(per_class_accuracy.values())
        })

        # Get top and bottom 5
        top_5 = performance_df.nlargest(5, 'Accuracy')
        bottom_5 = performance_df.nsmallest(5, 'Accuracy')

        # Overall accuracy for this set
        set_accuracy = np.mean(set_pred == set_true)

        # Save results
        with open(os.path.join(save_path, f'{name}_evaluation.txt'), 'w') as f:
            f.write(f"===== {name.upper()} CLASSES EVALUATION =====\n\n")
            f.write(f"Total Classes: {len(class_indices)}\n")
            f.write(f"Total Samples: {np.sum(mask)}\n")
            f.write(f"Overall Accuracy: {set_accuracy:.4f}\n\n")

            f.write("TOP 5 BEST CLASSIFIED CARS:\n")
            f.write("===========================\n")
            for _, row in top_5.iterrows():
                f.write(f"{row['Class']:<40} ({row['Accuracy']:.4f})\n")

            f.write("\nBOTTOM 5 WORST CLASSIFIED CARS:\n")
            f.write("==============================\n")
            for _, row in bottom_5.iterrows():
                f.write(f"{row['Class']:<40} ({row['Accuracy']:.4f})\n")

        return set_accuracy, len(class_indices), np.sum(mask)

    # Analyze both sets
    print("\nAnalyzing results...")
    stanford_acc, stanford_classes, stanford_samples = analyze_class_set(stanford_class_indices, "stanford")
    scraped_acc, scraped_classes, scraped_samples = analyze_class_set(scraped_class_indices, "scraped")

    # Print summary
    print("\nResults Summary:")
    print(f"Stanford Classes: {stanford_classes} classes, {stanford_samples} samples, {stanford_acc:.4f} accuracy")
    print(f"Scraped Classes: {scraped_classes} classes, {scraped_samples} samples, {scraped_acc:.4f} accuracy")

# Load model
print("Loading model...")
model_path = "/content/drive/My Drive/LocalCV/ResNet50_v2/car_classifier_resnet50_v2_final.h5"
loaded_model = load_model(model_path)

# Create save directory if it doesn't exist
save_path = "/content/drive/My Drive/LocalCV/ResNet50_v2"
os.makedirs(save_path, exist_ok=True)

# Evaluate the data
print("Evaluating test data...")
evaluate_with_class_types(loaded_model, test_generator, save_path)

print("\nEvaluation completed! Results saved to:")
print(f"{save_path}/stanford_evaluation.txt")
print(f"{save_path}/scraped_evaluation.txt")

Loading model...


Evaluating test data...


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


186/186 ━━━━━━━━━━━━━━━━━━━━ 23s 107ms/step

Analyzing results...

Results Summary:
Stanford Classes: 195 classes, 3264 samples, 0.6265 accuracy
Scraped Classes: 196 classes, 2668 samples, 0.3201 accuracy

Evaluation completed! Results saved to:
/content/drive/My Drive/LocalCV/ResNet50_v2/stanford_evaluation.txt
/content/drive/My Drive/LocalCV/ResNet50_v2/scraped_evaluation.txt
